In [1]:
# https://gym.openai.com/
# https://gym.openai.com/docs/
try:
    import gym
    
except ModuleNotFoundError as e:
    import sys
    !{sys.executable} -m pip install gym
    import gym
finally:
    from gym.envs.registration import register

  Stored in directory: C:\Users\15Z970-GA5BK\AppData\Local\pip\Cache\wheels\00\b8\b2\e5dfb6be621560717e719734293eee9fe3c66b668fdc334b1a
Successfully built gym
  Found existing installation: cloudpickle 0.8.0
    Uninstalling cloudpickle-0.8.0:
      Successfully uninstalled cloudpickle-0.8.0


spyder 3.3.3 requires pyqt5<=5.12; python_version >= "3", which is not installed.


In [2]:
# Gym 환경설정
# https://gym.openai.com/envs/ 에 들어가면 다양한 환경을 살펴볼 수 있음
# Frozen-Lake : https://gym.openai.com/envs/FrozenLake-v0/ 
register(
    id = 'FrozenLake-v1', 
    entry_point='gym.envs.toy_text:FrozenLakeEnv', 
    kwargs={
        # 'is_slippery': False 이면 Deteministic 환경.
        # 'is_slippery': True 이면 Stochastic 환경.
        'map_name' : '4x4', # 4x4 크기의 맵
        'is_slippery' : False # 미끄러질 가능성 없음 (나중에 다룰 것)   #내가 원하는대로 갈 확률 33% 
                              #is slippery가 false면 무조건 내가 원하는 대로 움직임
    }
)

In [3]:
env = gym.make("FrozenLake-v1")
env.render()


SFFF
FHFH
FFFH
HFFG


In [4]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [5]:
env.observation_space, env.action_space, env.observation_space.n, env.action_space.n

(Discrete(16), Discrete(4), 16, 4)

In [52]:
# LEFT = 0
# DOWN = 1
# RIGHT = 2
# UP = 3

# 임의로 방향 하나를 선택함.
action = env.action_space.sample()
action

0

In [53]:
observation, reward, done, info = env.step(action)
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


In [54]:
env.reset()

0

In [55]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [62]:
action = env.action_space.sample()
observation, reward, done, info = env.step(0)
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


In [63]:
print("위치: ", observation)
print("보상: ",reward)
print("Game Over?: ",done)
print("정보(확률): ", info) # 내가 의도한대로 갈 확률
print("행동: ", action)

위치:  5
보상:  0
Game Over?:  True
정보(확률):  {'prob': 1.0}
행동:  1


In [67]:
import numpy as np      ####  http://yujuwon.tistory.com/entry/NumPy
import random


# np.zeros((16, 4))
Q = np.zeros((env.observation_space.n, env.action_space.n))    # (16,4) : 4*4 map + 상하좌우 4개
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [68]:
# 횟수 초기화
num_episodes = 1000 #게임 총 몇번 할거야?
learning_rate = 0.3 #보상을 받았을 때 얼마의 비율로 업데이트를 할것인가
discount_rate = 0.99 #할인율: 바로 직전 뿐만 아니라 과거에 했떤 모든 경험들이 얼마나 큰 비중을 경험에 차지할 것인가

In [69]:
total_reward = 0.0
for epi in range(num_episodes):
    observation = env.reset() # 초기화
    
    episode_reward = 0.0
    
    #게임 시작점
    for t in range(100):
        # 1턴 실행 후의 위치
        current_state = observation
        
        # 학습을 빨리 하기 위해서 넣는 코드
        # 없으면 계속 LEFT로 가기 때문
        # 전체가 0이면 처리
        if not Q[current_state, :].any():
            action = env.action_space.sample()
        elif np.random.rand() < 0.5:
            # 랜덤 선택
            action = env.action_space.sample()
        else:
            # 최대값
            action = np.argmax(Q[current_state])
        
        observation, reward, done, info = env.step(action)
        
        # 성공 사례를 만들어야 측정이 가능함.
        # 현 위치의 최대값 = 성공 가능성
        # 이전 위치의 현 방향 선택 = 기존의 성공 가능성
        Q[current_state, action] += learning_rate * \ #최댓값 1
        (reward + discount_rate * \ #최솟값 -1
         np.max(Q[observation, :]) - \
         # reward가 1이면 observation은 0이 된다. 
         Q[current_state, action]) #최댓값 0.3
        #최댓값을 빼었기 때문에 기댓값
        if done:
            episode_reward +=reward
            break
    if epi % 500 == 0:
        print(epi, "번 진행중..")
    total_reward += episode_reward

0 번 진행중..
500 번 진행중..


In [70]:
total_reward/ num_episodes

0.364

In [73]:
Q
#랜덤으로 판을 돌다가 G에 골인하게 되면 그제서야 최초 학습이 적용됨. 
#G에 골인하면 뒤에서부터, 16번째에 골인이면 15번째의 선택을 학습하면서 거꾸로 스타트까지 올라감.
# 그렇게 학습률 적용


array([[0.94148015, 0.95099002, 0.95099005, 0.94148015],
       [0.94148015, 0.        , 0.96059601, 0.95099005],
       [0.95099005, 0.970299  , 0.95099005, 0.96059601],
       [0.96059601, 0.        , 0.95050301, 0.95098723],
       [0.94986082, 0.960596  , 0.        , 0.94148015],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9801    , 0.        , 0.96059601],
       [0.        , 0.        , 0.        , 0.        ],
       [0.9210285 , 0.        , 0.970299  , 0.87193565],
       [0.94998446, 0.9797854 , 0.9801    , 0.        ],
       [0.970299  , 0.99      , 0.        , 0.970299  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.97129604, 0.99      , 0.93666834],
       [0.9801    , 0.99      , 1.        , 0.9801    ],
       [0.        , 0.        , 0.        , 0.        ]])

In [74]:
Q.reshape(4,4,4)

array([[[0.94148015, 0.95099002, 0.95099005, 0.94148015],
        [0.94148015, 0.        , 0.96059601, 0.95099005],
        [0.95099005, 0.970299  , 0.95099005, 0.96059601],
        [0.96059601, 0.        , 0.95050301, 0.95098723]],

       [[0.94986082, 0.960596  , 0.        , 0.94148015],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.9801    , 0.        , 0.96059601],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.9210285 , 0.        , 0.970299  , 0.87193565],
        [0.94998446, 0.9797854 , 0.9801    , 0.        ],
        [0.970299  , 0.99      , 0.        , 0.970299  ],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.97129604, 0.99      , 0.93666834],
        [0.9801    , 0.99      , 1.        , 0.9801    ],
        [0.        , 0.        , 0.        , 0.        ]]])

In [75]:
# 횟수 초기화
num_episodes = 1000

In [81]:
total_reward = 0.0
for i_episode in range(num_episodes):
    observation = env.reset()
    
    episode_reward = 0.0
    for t in range(100):
        #1턴 후의 위치
        current_state = observation
        
        # Q값이 최대가 되는 행동을 선택함
        action = np.argmax(Q[current_state])
        
        # 1턴 실행
        observation, reward, done, info = env.step(action)
        
        #종료
        if done:
            episode_reward += reward
            
    # 총 reward
    total_reward += episode_reward

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.

In [77]:
print(total_reward)

1000.0


In [78]:
total_reward / num_episodes

1.0

# 2. Stochastic 환경.

In [82]:

# Gym 환경설정
# https://gym.openai.com/envs/ 에 들어가면 다양한 환경을 살펴볼 수 있음
# Frozen-Lake : https://gym.openai.com/envs/FrozenLake-v0/ 
register(
    id = 'FrozenLake-v2', 
    entry_point='gym.envs.toy_text:FrozenLakeEnv', 
    kwargs={
        # 'is_slippery': False 이면 Deteministic 환경.
        # 'is_slippery': True 이면 Stochastic 환경.
        'map_name' : '4x4', # 4x4 크기의 맵
        'is_slippery' : True # 미끄러질 가능성 있음
        
    }
)

In [98]:
env = gym.make("FrozenLake-v2")
env.render()


SFFF
FHFH
FFFH
HFFG


In [99]:
env.reset()

0

In [111]:
action = env.action_space.sample()
observation, reward, done, info = env.step(action)
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


In [86]:
print("위치: ", observation)
print("보상: ",reward)
print("Game Over?: ",done)
print("정보(확률): ", info) # 내가 의도한대로 갈 확률
print("행동: ", action)

위치:  1
보상:  0.0
Game Over?:  False
정보(확률):  {'prob': 0.3333333333333333}
행동:  1


In [87]:
import numpy as np
# np.zeros((16, 4))
Q = np.zeros((env.observation_space.n, env.action_space.n))    # (16,4) : 4*4 map + 상하좌우 4개
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [119]:
# 횟수 초기화
num_episodes = 100000
learning_rate = 0.3
discount_rate = 0.99

In [120]:
total_reward = 0.0
recent_reward = []
recent_size = 100

for epi in range(num_episodes):
    observation = env.reset() # 초기화
    
    episode_reward = 0.0
    
    for t in range(100):
        # 1턴 실행 후의 위치
        current_state = observation
        
        # 학습을 빨리 하기 위해서 넣는 코드
        # 없으면 계속 LEFT로 가기 때문
        # 전체가 0이면 처리
        
        # if not Q[current_state, :].any():
        # action = env.action_space.sample()
        if np.random.rand() < 0.1: # 변경 된걸 확인하세요.
            # 랜덤 선택
            action = env.action_space.sample()
        else:
            # 최대값
            action = np.argmax(Q[current_state])
        
        observation, reward, done, info = env.step(action)
        
        # 성공 사례를 만들어야 측정이 가능함.
        # 현 위치의 최대값 = 성공 가능성
        # 이전 위치의 현 방향 선택 = 기존의 성공 가능성
        Q[current_state, action] += learning_rate * (reward + discount_rate * np.max(Q[observation, :]) - Q[current_state, action]) 
            #                        0.3 - 내결정 * 0.3
           #내 결정 = 내 결정 + 0.3 -0.3(내결정)
        if done:
            episode_reward +=reward
            break
    if epi % 500 == 0:
        print(epi, "번 진행중..")
    total_reward += episode_reward
    
    recent_reward.append(episode_reward)
    if len(recent_reward) >= recent_size:
        # 최근 100개에 대해서 검사
        recent_reward.pop(0)
        
        if sum(recent_reward) / recent_size > 0.7:
            print("0.7이 넘었습니다. 멈추겠습니다.")
            break

0 번 진행중..
500 번 진행중..
1000 번 진행중..
1500 번 진행중..
2000 번 진행중..
2500 번 진행중..
3000 번 진행중..
3500 번 진행중..
4000 번 진행중..
4500 번 진행중..
5000 번 진행중..
5500 번 진행중..
6000 번 진행중..
6500 번 진행중..
7000 번 진행중..
7500 번 진행중..
8000 번 진행중..
8500 번 진행중..
9000 번 진행중..
9500 번 진행중..
10000 번 진행중..
10500 번 진행중..
11000 번 진행중..
11500 번 진행중..
12000 번 진행중..
12500 번 진행중..
13000 번 진행중..
13500 번 진행중..
14000 번 진행중..
14500 번 진행중..
15000 번 진행중..
15500 번 진행중..
16000 번 진행중..
16500 번 진행중..
17000 번 진행중..
17500 번 진행중..
18000 번 진행중..
18500 번 진행중..
19000 번 진행중..
19500 번 진행중..
20000 번 진행중..
20500 번 진행중..
21000 번 진행중..
21500 번 진행중..
22000 번 진행중..
22500 번 진행중..
23000 번 진행중..
23500 번 진행중..
24000 번 진행중..
24500 번 진행중..
25000 번 진행중..
25500 번 진행중..
26000 번 진행중..
26500 번 진행중..
27000 번 진행중..
27500 번 진행중..
28000 번 진행중..
28500 번 진행중..
29000 번 진행중..
29500 번 진행중..
30000 번 진행중..
30500 번 진행중..
31000 번 진행중..
31500 번 진행중..
32000 번 진행중..
32500 번 진행중..
33000 번 진행중..
33500 번 진행중..
34000 번 진행중..
34500 번 진행중..
35000 번 진행중..
35500 번 진행중..
36000 번 진행중..
36

In [126]:
print("위치: ", observation)
print("보상: ", reward)
print("Game Over?: ",done)
print("정보(확률): ", info) # 내가 의도한대로 갈 확률
print("행동: ", action)

위치:  12
보상:  0.0
Game Over?:  True
정보(확률):  {'prob': 0.3333333333333333}
행동:  0


In [121]:
total_reward/ num_episodes

0.33052

In [122]:
sum(recent_reward)/recent_size

0.37

In [123]:
Q

array([[0.54563052, 0.53520556, 0.53789376, 0.53006509],
       [0.48143402, 0.23540841, 0.41164338, 0.51543244],
       [0.44502576, 0.34403206, 0.42390005, 0.47301338],
       [0.28290285, 0.15411191, 0.26094122, 0.46396364],
       [0.567145  , 0.50290285, 0.14548786, 0.47285774],
       [0.        , 0.        , 0.        , 0.        ],
       [0.46247832, 0.1603602 , 0.14698439, 0.08120084],
       [0.        , 0.        , 0.        , 0.        ],
       [0.31468133, 0.42458191, 0.42681909, 0.59623797],
       [0.24919672, 0.64846043, 0.39465703, 0.52345708],
       [0.33927144, 0.62277014, 0.2613628 , 0.19848275],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.45746611, 0.45017864, 0.71341276, 0.66585751],
       [0.77280702, 0.77579568, 0.76395012, 0.74593983],
       [0.        , 0.        , 0.        , 0.        ]])

# Q테이블로 테스트하기

In [127]:
# 횟수 초기화
num_episodes = 1000

total_reward = 0.0
for i_episode in range(num_episodes):
    observation = env.reset()
    
    episode_reward = 0.0
    for t in range(100):
        #1턴 후의 위치
        current_state = observation
        
        # Q값이 최대가 되는 행동을 선택함
        action = np.argmax(Q[current_state])
        
        # 1턴 실행
        observation, reward, done, info = env.step(action)
        
        #종료
        if done:
            episode_reward += reward
            
    # 총 reward
    total_reward += episode_reward

In [128]:
print(total_reward)

618.0


# DQN

In [132]:
import gym

import numpy as np

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

# Stochastic 환경.
env = gym.make('FrozenLake-v2')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(16, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 64)
        self.fc6 = nn.Linear(64, 64)
        self.fc7 = nn.Linear(64, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        
        return x

In [133]:
model = Net()

In [134]:

def onehot2tensor(state):
    tmp = np.zeros(16)
    tmp[state] = 1
    vector = np.array(tmp, dtype='float32')
    tensor = torch.from_numpy(vector).float()
    return tensor

def applymodel(tensor):
    output_tensor = model(tensor)
    output_array = output_tensor.data.numpy()
    return output_tensor, output_array

In [136]:
# cost function을 사용했으나 어느 지점부터 변화가 없음

total_reward = 0.0
num_episodes = 5000
discount_rate = 0.99

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.01)

for i_episode in range(num_episodes):
    observation = env.reset()
    
    # 현재 게임 reward
    episode_reward = 0.0
    
    # 누적 오차
    total_loss = 0.0
    
    for t in range(100):
        current_state = observation
        
        # 경사 초기화
        optimizer.zero_grad()
        
        # 현위치 onehot encoding 
        current_tensor = onehot2tensor(current_state)

        if (i_episode+1) % 500 == 0:
            print(current_tensor)
            current_output_tensor, current_output_array = applymodel(current_tensor)
            print(current_output_tensor)
            # print("{}번: total_loss: {}, total_reward: {} ".format(i_episode+1, total_loss, total_reward))
            
            
        # 모형에 입력
        current_output_tensor, current_output_array = applymodel(current_tensor)
        # 행동 선택
        if np.random.rand() < 0.1:
            #무작위 선택
            action = env.action_space.sample()
        else:
            # Q값이 최대가 되도록
            action = np.argmax(current_output_array)

        ## 1턴 실행
        observation, reward, done, info = env.step(action)
        
        # onehot 벡터 텐서로 변환
        observation_tensor = onehot2tensor(observation)
        
        # 모형에 입력
        observation_output_tensor, observation_output_array = applymodel(observation_tensor)
        
        # Q값 업데이트
        # y값
        q = reward + discount_rate * np.max(observation_output_array) # reward는 마지막 value를 위해 더함
        
        # 기존 보드의 q값 복사
        q_array = np.copy(current_output_array) # 이전 값
        q_array[action] = q
        q_variable = torch.from_numpy(q_array)
        
        # 오차 계산 - q_array와 q
        # 현재 아웃풋 값과, 기존 q어레이
        # y값은 올바른 선택일 때의 최대값,
        # q^은 현재 선택의 결과
        
        # current_output_tensor: 현재 측정값
        # q_variable: 최고의 선택을 했을 때 y값
        loss = criterion(current_output_tensor, q_variable)
        
        # 역전파
        loss.backward()
        
        # 웨이트 업데이트
        optimizer.step()
        
        # 오차 누적 계산
        total_loss += loss.item()
        
        if done:
            # 종료
            episode_reward += reward
            
            break
    total_reward += episode_reward
    
    # 누적오차 및 보상률
    if (i_episode+1) % 500 == 0:
        print("{}번: total_loss: {}, total_reward: {} ".format(i_episode+1, total_loss, total_reward))

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([45.0314, 47.9362, 45.1100, 43.5702], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([46.4968, 49.5300, 46.5825, 45.0658], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([48.7093, 51.9264, 48.8049, 47.2945], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([49.8167, 53.1310, 49.9177, 48.4261], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([50.6564, 54.0452, 50.7616, 49.2872], grad_fn=<AddBackward0>)
500번: total_loss: 0.4183310456573963, total_reward: 18.0 
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.6698, 0.5731, 0.6698, 0.6697], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.6697, 0.573

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5175, 0.5174, 0.4700, 0.5177], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5168, 0.5168, 0.4707, 0.5171], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5162, 0.5163, 0.4714, 0.5163], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5156, 0.5158, 0.4720, 0.5155], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5151, 0.5152, 0.4725, 0.5148], grad_fn=<AddBackward0>)
3000번: total_loss: 0.0012987912109565514, total_reward: 90.0 
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5050, 0.5062, 0.5060, 0.5078], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5041, 0.5057, 0.5054, 0.507

In [137]:
total_reward = 0.0
num_episodes = 1000

for i_episode in range(num_episodes):
    observation = env.reset()
    
    # 현재 게임 reward
    episode_reward = 0.0
    
    
    for t in range(100):
        current_state = observation
        
        
        # 현위치 onehot encoding 
        current_tensor = onehot2tensor(current_state)
        # 모형에 입력
        current_output_tensor, current_output_array = applymodel(current_tensor)
        
        # Q테이블에서 제일 큰거 선택
        action = np.argmax(current_output_array)

        ## 1턴 실행
        observation, reward, done, info = env.step(action)
        
        if done:
            # 종료
            episode_reward += reward
            break
            
    total_reward += episode_reward
    
    # 누적오차 및 보상률
    if (i_episode+1) % 500 == 0:
        print("{}번:".format(i_episode+1))
        print(total_reward)
        print("-"*30)

500번:
38.0
------------------------------
1000번:
63.0
------------------------------


In [138]:
print(total_reward)  
print(total_reward/1000)

63.0
0.063
